---
title: Surface Ocean heat from CESM2 LENS data
author: Harsha R. Hampapura
---

## Calculate surface ocean heat content using CESM2 LENS data on a Jetstream 2 exosphere instance
### Table of Contents
- [Section 1: Introduction](#Section-1:-Introduction) 
- [Section 2: Select Dask Cluster](#Section-2:-Select-Dask-Cluster) 
- [Section 3: Data Loading](#Section-3:-Data-Loading) 
- [Section 4: Data Analysis](#Section-4:-Data-Analysis)  

- This notebook is adapted from the NCAR gallery in the Pangeo collection
- https://gallery.pangeo.io/repos/NCAR/notebook-gallery/notebooks/Run-Anywhere/Ocean-Heat-Content/OHC_tutorial.html

## Section 1: Introduction
### Input Data Access

- This notebook illustrates how to compute surface ocean heat content using potential temperature data from CESM2 Large Ensemble Dataset (https://www.cesm.ucar.edu/community-projects/lens2) hosted on NCAR's GDEX.
- This data is open access and is accessed via OSDF

In [1]:
# Imports
import intake
import numpy as np
import pandas as pd
import xarray as xr
# import seaborn as sns
import re
import os
import matplotlib.pyplot as plt
import dask
from dask.distributed import LocalCluster
import cf_units as cf

In [2]:
init_year0  = '1991'
init_year1  = '2020'
final_year0 = '2071'
final_year1 = '2100'

In [3]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")

In [4]:
# Set up your sratch folder path
# username       = os.environ["USER"]
# scratch  = "/" + username
# print(scratch)
#
catalog_url = 'https://osdata.gdex.ucar.edu/d010092/catalogs/d010092-osdf.json'

## Section 2: Set up Dask Cluster
- Setting up a dask cluster. 
- The default will be LocalCluster as that can run on any system.

In [5]:
cluster = LocalCluster()
client = cluster.get_client()

In [6]:
# Scale the local cluster
n_workers = 5
cluster.scale(n_workers)
cluster

LocalCluster(835f31fc, 'tcp://127.0.0.1:45379', workers=4, threads=4, memory=14.63 GiB)

## Section 3: Data Loading
- Load CESM2 LENS zarr data from GDEX using an intake-ESM catalog
- For more details regarding the dataset. See, https://gdex.ucar.edu/datasets/d010092/#

In [7]:
cesm_cat = intake.open_esm_datastore(catalog_url)
cesm_cat

,unique
,322
variable,54
long_name,52
component,4
experiment,2
forcing_variant,2
frequency,3
vertical_levels,4
spatial_domain,3
units,21


In [8]:
# cesm_cat.df['variable'].values

In [13]:
%pip show zarr

Name: zarr
Version: 3.1.5
Summary: An implementation of chunked, compressed, N-dimensional arrays for Python
Home-page: https://github.com/zarr-developers/zarr-python
Author: 
Author-email: Alistair Miles <alimanfoo@googlemail.com>
License-Expression: MIT
Location: /home/exouser/.conda/envs/osdf/lib/python3.11/site-packages
Requires: donfig, google-crc32c, numcodecs, numpy, packaging, typing-extensions
Required-by: intake-esm, kerchunk
Note: you may need to restart the kernel to use updated packages.


In [10]:
cesm_temp = cesm_cat.search(variable ='TEMP', frequency ='monthly',experiment='historical')
cesm_temp

,unique
column_0,1
variable,1
long_name,1
component,1
experiment,1
forcing_variant,1
frequency,1
vertical_levels,1
spatial_domain,1
units,1


In [11]:
cesm_temp.df['path'].values

<ArrowExtensionArray>
['osdf:///ncar-gdex/d010092/ocn/monthly/cesm2LE-historical-cmip6-TEMP.zarr']
Length: 1, dtype: large_string[pyarrow]

:::{note}: Important Note!
Because our environment has zarr version >= 3 and the zarr stores we are trying to open were created using zarr version <3,
we need to force the use of zarr version 2 by passing keyword arguments to the to_dataset_dict() function
:::

In [12]:
dsets_cesm = cesm_temp.to_dataset_dict(xarray_open_kwargs={'engine':'zarr','backend_kwargs':{'consolidated': True,'zarr_format': 2}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


<div><progress max="1" value="0"></progress> 0.00% [0/1 00:00&lt;?]</div>

No working cache found
No working cache found
No working cache found
2026-02-03 23:48:06,290 - distributed.worker - ERROR - Compute Failed
Key:       _delayed_open_ds-fbcb1459-d750-4ddc-9dc7-20f987206a46
State:     executing
Task:  <Task '_delayed_open_ds-fbcb1459-d750-4ddc-9dc7-20f987206a46' _delayed_open_ds(..., ...)>
Exception: 'NoAvailableSource()'
Traceback: '  File "/home/exouser/.conda/envs/osdf/lib/python3.11/site-packages/intake_esm/source.py", line 67, in _delayed_open_ds\n    return _open_dataset(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/exouser/.conda/envs/osdf/lib/python3.11/site-packages/intake_esm/source.py", line 109, in _open_dataset\n    ds = xr.open_dataset(url, **xarray_open_kwargs)\n         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/exouser/.conda/envs/osdf/lib/python3.11/site-packages/xarray/backends/api.py", line 606, in open_dataset\n    backend_ds = backend.open_dataset(\n                 ^^^^^^^^^^^^^^^^^^^^^\n 

ESMDataSourceError: Failed to load dataset with key='ocn.historical.monthly.cmip6'
                 You can use `cat['ocn.historical.monthly.cmip6'].df` to inspect the assets/files for this key.
                 

In [ ]:
cesm_temp.keys()

In [ ]:
historical       = dsets_cesm['ocn.historical.monthly.cmip6']
# future_smbb      = dsets_cesm['ocn.ssp370.monthly.smbb']
# future_cmip6     = dsets_cesm['ocn.ssp370.monthly.cmip6']

In [ ]:
# %%time
# merge_ds_cmip6 = xr.concat([historical, future_cmip6], dim='time')
# merge_ds_cmip6 = merge_ds_cmip6.dropna(dim='member_id')

In [ ]:
historical

#### Change units

In [ ]:
orig_units = cf.Unit(historical.z_t.attrs['units'])
orig_units

In [ ]:
def change_units(ds, variable_str, variable_bounds_str, target_unit_str):
    orig_units = cf.Unit(ds[variable_str].attrs['units'])
    target_units = cf.Unit(target_unit_str)
    variable_in_new_units = xr.apply_ufunc(orig_units.convert, ds[variable_bounds_str], target_units, dask='parallelized', output_dtypes=[ds[variable_bounds_str].dtype])
    return variable_in_new_units

In [ ]:
historical['z_t']

In [ ]:
depth_levels_in_m = change_units(historical, 'z_t', 'z_t', 'm')
hist_temp_in_degK = change_units(historical, 'TEMP', 'TEMP', 'degK')
# fut_cmip6_temp_in_degK = change_units(future_cmip6, 'TEMP', 'TEMP', 'degK')
# fut_smbb_temp_in_degK = change_units(future_smbb, 'TEMP', 'TEMP', 'degK')
#
hist_temp_in_degK  = hist_temp_in_degK.assign_coords(z_t=("z_t", depth_levels_in_m['z_t'].data))
hist_temp_in_degK["z_t"].attrs["units"] = "m"
hist_temp_in_degK

In [ ]:
depth_levels_in_m.isel(z_t=slice(0, -1))

In [ ]:
#Compute depth level deltas using z_t levels
depth_level_deltas = depth_levels_in_m.isel(z_t=slice(1, None)).values - depth_levels_in_m.isel(z_t=slice(0, -1)).values
# Optionally, if you want to keep it as an xarray DataArray, re-wrap the result
depth_level_deltas = xr.DataArray(depth_level_deltas, dims=["z_t"], coords={"z_t": depth_levels_in_m.z_t.isel(z_t=slice(0, -1))})
depth_level_deltas                                                                                        

## Section 4: Data Analysis 
#### Compute Ocean Heat content for ocean surface
- Ocean surface is considered to be the top 100m
- The formula for this is: $$ H = \rho C \int_0^z T(z) dz $$


Where H is ocean heat content, the value we are trying to calculate,

$\rho$ is the density of sea water, $1026 kg/m^3$  ,

$C$ is the specific heat of sea water, $3990 J/(kg K)$  ,

$z$ is the depth limit of the calculation in meters,

and $T(z)$ is the temperature at each depth in degrees Kelvin.

In [ ]:
def calc_ocean_heat(delta_level, temperature):
    rho = 1026 #kg/m^3
    c_p = 3990 #J/(kg K)
    weighted_temperature = delta_level * temperature
    heat = weighted_temperature.sum(dim="z_t")*rho*c_p
    return heat

In [ ]:
# Remember that the coordinate z_t still has values in cm
hist_temp_ocean_surface = hist_temp_in_degK.where(hist_temp_in_degK['z_t'] < 1e4,drop=True)
hist_temp_ocean_surface

In [ ]:
depth_level_deltas_surface = depth_level_deltas.where(depth_level_deltas['z_t'] <1e4, drop= True)
depth_level_deltas_surface

In [ ]:
hist_ocean_heat = calc_ocean_heat(depth_level_deltas_surface,hist_temp_ocean_surface)
hist_ocean_heat

### Plot Ocean Heat

In [ ]:
%%time
# Jan, 1850 average over all memebers
# hist_ocean_avgheat = hist_ocean_heat.mean('member_id')
hist_ocean_avgheat = hist_ocean_heat.isel({'time':[0,-12]}).mean('member_id')
hist_ocean_avgheat

In [ ]:
%%time
hist_ocean_avgheat.isel(time=0).plot()

In [ ]:
%%time
#Plot ocean heat for Jan 2014
hist_ocean_avgheat.isel(time=1).plot()

### Has the surface ocean heat content increased with time for January ? (Due to Global Warming!)

In [ ]:
hist_ocean_avgheat_ano = hist_ocean_avgheat.isel(time=1) - hist_ocean_avgheat.isel(time=0)

In [ ]:
%%time
hist_ocean_avgheat_ano.plot()

In [ ]:
cluster.close()